# ES → BBF Migration Data Analysis (v3)

This notebook analyzes all data required for the ES to BBF Salesforce migration.

## Version 3 Changes
- Include OA (Accepted) orders in addition to CL (Closed) - both must pass billing date criteria
- Data Quality updated: OSS `bill_start_date` can be used when SF `Service_Start_Date__c` is missing
- Work Orders enriched with data from `workorders.workorders` table

## Driving Principle
**Everything is driven from Active ES Orders that are ACTUALLY BILLING in OSS** - we migrate only the data needed to support truly active services.

## Filter Pipeline
1. Status IN ('Activated', 'Suspended (Late Payment)', 'Disconnect in Progress')
2. Project_Group__c NOT LIKE '%PA MARKET DECOM%'
3. Service_Order_Record_Type__c = 'Service Order Agreement' (excludes Work Orders)
4. OSS Actively Billing:
   - order_state_cd IN ('CL', 'OA')
   - bill_start_date <= today
   - bill_end_date IS NULL or > today
5. Has BBF BAN mapping

In [1]:
# === SETUP & IMPORTS ===

import sys
import pandas as pd
import psycopg2
from psycopg2.extras import RealDictCursor
from simple_salesforce import Salesforce
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils.dataframe import dataframe_to_rows
from datetime import datetime
from collections import defaultdict

print(f"Python: {sys.executable}")
print(f"Pandas: {pd.__version__}")
print("✅ Imports successful")

Python: C:\Users\vjero\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe
Pandas: 2.2.3
✅ Imports successful


In [2]:
# === CONFIGURATION ===

# ES (Source) Salesforce Credentials
ES_USERNAME = "sfdcapi@everstream.net"
ES_PASSWORD = "pV4CAxns8DQtJsBq!"
ES_TOKEN = "r1uoYiusK19RbrflARydi86TA"
ES_DOMAIN = "login"  # 'login' for production

# OSS Database Credentials
OSS_HOST = "pg01.comlink.net"
OSS_PORT = "5432"
OSS_DB = "GLC"
OSS_USER = "oss_server"
OSS_PASSWORD = "3wU3uB28X?!r2?@ebrUg"

# Active Order Status Filter
ACTIVE_STATUSES = ["Activated", "Suspended (Late Payment)", "Disconnect in Progress"]

# Record Type Filter
VALID_RECORD_TYPE = "Service Order Agreement"

# PA Market Decom Exclusion
PA_DECOM_FILTER = "PA MARKET DECOM"

# OSS Order States that qualify as "actively billing"
ACTIVE_OSS_STATES = ["CL", "OA"]  # Closed and Accepted

# OSS Order States Reference
OSS_ORDER_STATES = {
    "CL": "Closed (Active/Billing)",
    "OA": "Accepted",
    "OS": "Submitted",
    "OC": "Created",
    "PN": "Pending",
    "CA": "Cancelled",
    "OR": "Rejected",
    "OV": "Validated (Disabled)",
}

# OSS Work Order Types Reference
WORKORDER_TYPES = {
    "IT": "Professional Services",
    "MR": "Maintenance/Repair",
    "OS": "Other Service",
    "VS": "Voice Service",
}

# OSS Work Order States Reference
WORKORDER_STATES = {
    "CA": "Cancelled",
    "CL": "Closed",
    "OA": "Accepted",
    "PN": "Pending",
}

# Output Configuration
TIMESTAMP = datetime.now().strftime("%Y%m%d_%H%M%S")
OUTPUT_FILE = f"es_bbf_migration_analysis_v3_{TIMESTAMP}.xlsx"

print("📋 Configuration loaded")
print(f"   Active Statuses: {ACTIVE_STATUSES}")
print(f"   Record Type: {VALID_RECORD_TYPE}")
print(f"   Excluding: Project_Group__c LIKE '%{PA_DECOM_FILTER}%'")
print(f"   OSS Active States: {ACTIVE_OSS_STATES}")
print(f"   Output: {OUTPUT_FILE}")

📋 Configuration loaded
   Active Statuses: ['Activated', 'Suspended (Late Payment)', 'Disconnect in Progress']
   Record Type: Service Order Agreement
   Excluding: Project_Group__c LIKE '%PA MARKET DECOM%'
   OSS Active States: ['CL', 'OA']
   Output: es_bbf_migration_analysis_v3_20260107_203703.xlsx


In [3]:
# === CONNECT TO ES SALESFORCE ===

print("=" * 80)
print("CONNECTING TO ES SALESFORCE")
print("=" * 80)

print("\n🔌 Connecting to ES...")
es_sf = Salesforce(
    username=ES_USERNAME,
    password=ES_PASSWORD,
    security_token=ES_TOKEN,
    domain=ES_DOMAIN,
)
print(f"✅ Connected to ES: {es_sf.sf_instance}")

CONNECTING TO ES SALESFORCE

🔌 Connecting to ES...
✅ Connected to ES: everstream.my.salesforce.com


In [4]:
# === CONNECT TO OSS DATABASE ===

print("\n" + "=" * 80)
print("CONNECTING TO OSS DATABASE")
print("=" * 80)

print("\n🔌 Connecting to OSS...")
oss_conn = psycopg2.connect(
    dbname=OSS_DB,
    user=OSS_USER,
    password=OSS_PASSWORD,
    host=OSS_HOST,
    port=OSS_PORT,
)
print(f"✅ Connected to OSS: {OSS_HOST}/{OSS_DB}")


CONNECTING TO OSS DATABASE

🔌 Connecting to OSS...
✅ Connected to OSS: pg01.comlink.net/GLC


In [5]:
# === STEP 1: QUERY ALL ACTIVE ORDERS ===

print("\n" + "=" * 80)
print("STEP 1: QUERYING ACTIVE ORDERS")
print("=" * 80)

status_filter = "','".join(ACTIVE_STATUSES)

orders_query = f"""
SELECT 
    Id, 
    Name,
    Service_ID__c,
    Status,
    AccountId,
    Account.Name,
    Billing_Invoice__c,
    Address_A__c,
    Address_Z__c,
    Node__c,
    OpportunityId,
    Service_Start_Date__c,
    Service_End_Date__c,
    Service_Provided__c,
    SOF_MRC__c,
    OSS_Order__c,
    OSS_Service_ID__c,
    Vendor_Circuit_ID__c,
    Primary_Product_Family__c,
    Primary_Product_Name__c,
    Project_Group__c,
    Service_Order_Record_Type__c,
    CreatedDate,
    LastModifiedDate
FROM Order
WHERE Status IN ('{status_filter}')
ORDER BY Service_ID__c
"""

print("Querying all orders with active statuses...")
result = es_sf.query_all(orders_query)
orders_raw = result["records"]

# Flatten results
all_orders = []
for order in orders_raw:
    all_orders.append(
        {
            "Id": order["Id"],
            "Name": order.get("Name"),
            "Service_ID__c": order.get("Service_ID__c"),
            "Status": order["Status"],
            "AccountId": order.get("AccountId"),
            "Account_Name": order["Account"]["Name"] if order.get("Account") else None,
            "Billing_Invoice__c": order.get("Billing_Invoice__c"),
            "Address_A__c": order.get("Address_A__c"),
            "Address_Z__c": order.get("Address_Z__c"),
            "Node__c": order.get("Node__c"),
            "OpportunityId": order.get("OpportunityId"),
            "Service_Start_Date__c": order.get("Service_Start_Date__c"),
            "Service_End_Date__c": order.get("Service_End_Date__c"),
            "Service_Provided__c": order.get("Service_Provided__c"),
            "SOF_MRC__c": order.get("SOF_MRC__c"),
            "OSS_Order__c": order.get("OSS_Order__c"),
            "OSS_Service_ID__c": order.get("OSS_Service_ID__c"),
            "Vendor_Circuit_ID__c": order.get("Vendor_Circuit_ID__c"),
            "Primary_Product_Family__c": order.get("Primary_Product_Family__c"),
            "Primary_Product_Name__c": order.get("Primary_Product_Name__c"),
            "Project_Group__c": order.get("Project_Group__c"),
            "Service_Order_Record_Type__c": order.get("Service_Order_Record_Type__c"),
            "CreatedDate": order.get("CreatedDate"),
            "LastModifiedDate": order.get("LastModifiedDate"),
        }
    )

all_orders_df = pd.DataFrame(all_orders)
print(f"\n✅ Total orders with active status: {len(all_orders_df):,}")

# Show record type breakdown
print(f"\n📊 Record Type Breakdown:")
print(all_orders_df["Service_Order_Record_Type__c"].value_counts(dropna=False))


STEP 1: QUERYING ACTIVE ORDERS
Querying all orders with active statuses...

✅ Total orders with active status: 17,972

📊 Record Type Breakdown:
Service_Order_Record_Type__c
Service Order Agreement    14058
Work Order                  3914
Name: count, dtype: int64


In [6]:
# === STEP 2: FILTER - PA MARKET DECOM ===

print("\n" + "=" * 80)
print("STEP 2: FILTER - PA MARKET DECOM")
print("=" * 80)

# Separate PA MARKET DECOM orders
pa_decom_mask = (
    all_orders_df["Project_Group__c"]
    .fillna("")
    .str.contains(PA_DECOM_FILTER, case=False)
)
excluded_pa_decom_df = all_orders_df[pa_decom_mask].copy()
orders_after_pa_filter = all_orders_df[~pa_decom_mask].copy()

print(f"\n📊 PA MARKET DECOM Filter:")
print(f"   Before: {len(all_orders_df):,}")
print(f"   ❌ Excluded (PA MARKET DECOM): {len(excluded_pa_decom_df):,}")
print(f"   ✅ Remaining: {len(orders_after_pa_filter):,}")


STEP 2: FILTER - PA MARKET DECOM

📊 PA MARKET DECOM Filter:
   Before: 17,972
   ❌ Excluded (PA MARKET DECOM): 887
   ✅ Remaining: 17,085


In [7]:
# === STEP 3: FILTER - RECORD TYPE (Service Order Agreement only) ===

print("\n" + "=" * 80)
print("STEP 3: FILTER - RECORD TYPE")
print("=" * 80)

# Keep only Service Order Agreement
record_type_mask = (
    orders_after_pa_filter["Service_Order_Record_Type__c"] == VALID_RECORD_TYPE
)
excluded_work_orders_df = orders_after_pa_filter[~record_type_mask].copy()
orders_after_rt_filter = orders_after_pa_filter[record_type_mask].copy()

print(f"\n📊 Record Type Filter:")
print(f"   Before: {len(orders_after_pa_filter):,}")
print(f"   ❌ Excluded (Not '{VALID_RECORD_TYPE}'): {len(excluded_work_orders_df):,}")
print(f"   ✅ Remaining: {len(orders_after_rt_filter):,}")

# Show breakdown of what was excluded
if len(excluded_work_orders_df) > 0:
    print(f"\n   Excluded Record Types:")
    print(
        excluded_work_orders_df["Service_Order_Record_Type__c"].value_counts(
            dropna=False
        )
    )


STEP 3: FILTER - RECORD TYPE

📊 Record Type Filter:
   Before: 17,085
   ❌ Excluded (Not 'Service Order Agreement'): 3,477
   ✅ Remaining: 13,608

   Excluded Record Types:
Service_Order_Record_Type__c
Work Order    3477
Name: count, dtype: int64


In [8]:
# === STEP 3B: ENRICH WORK ORDERS WITH OSS DATA ===

print("\n" + "=" * 80)
print("STEP 3B: ENRICHING WORK ORDERS WITH OSS DATA")
print("=" * 80)

if len(excluded_work_orders_df) > 0:
    # Get work orders with OSS_Order__c (maps to workorder_id)
    has_wo_oss = excluded_work_orders_df["OSS_Order__c"].notna() & (
        excluded_work_orders_df["OSS_Order__c"] != ""
    )
    work_orders_with_oss = excluded_work_orders_df[has_wo_oss].copy()

    print(f"\n   Work Orders with OSS_Order__c: {len(work_orders_with_oss):,}")
    print(f"   Work Orders without OSS_Order__c: {(~has_wo_oss).sum():,}")

    if len(work_orders_with_oss) > 0:
        workorder_ids = (
            work_orders_with_oss["OSS_Order__c"].dropna().astype(int).unique().tolist()
        )
        print(f"   Unique workorder IDs to query: {len(workorder_ids):,}")

        # Query workorders.workorders
        chunk_size = 5000
        oss_workorders = []

        for i in range(0, len(workorder_ids), chunk_size):
            chunk = workorder_ids[i : i + chunk_size]
            ids_str = ",".join(str(x) for x in chunk)

            wo_query = f"""
            SELECT 
                workorder_id,
                order_nm,
                order_id,
                workorder_type_cd,
                workorder_state_cd,
                description,
                start_date,
                end_date,
                disabled
            FROM workorders.workorders
            WHERE workorder_id IN ({ids_str})
              AND disabled = 'infinity'
            """

            with oss_conn.cursor(cursor_factory=RealDictCursor) as cur:
                cur.execute(wo_query)
                rows = cur.fetchall()
                oss_workorders.extend([dict(row) for row in rows])

            print(f"   Chunk {i//chunk_size + 1}: Retrieved {len(rows)} workorders")

        if len(oss_workorders) > 0:
            oss_wo_df = pd.DataFrame(oss_workorders)

            # Add descriptions
            oss_wo_df["workorder_type_desc"] = oss_wo_df["workorder_type_cd"].map(
                WORKORDER_TYPES
            )
            oss_wo_df["workorder_state_desc"] = oss_wo_df["workorder_state_cd"].map(
                WORKORDER_STATES
            )

            # Merge with excluded work orders
            excluded_work_orders_df["OSS_Order_ID"] = pd.to_numeric(
                excluded_work_orders_df["OSS_Order__c"], errors="coerce"
            )
            excluded_work_orders_df = excluded_work_orders_df.merge(
                oss_wo_df, left_on="OSS_Order_ID", right_on="workorder_id", how="left"
            )

            print(
                f"\n✅ Enriched {oss_wo_df['workorder_id'].notna().sum():,} work orders with OSS data"
            )
        else:
            print("\n⚠️ No active workorders found in OSS")
    else:
        print("\n⚠️ No work orders have OSS_Order__c populated")
else:
    print("\n⚠️ No work orders to enrich")


STEP 3B: ENRICHING WORK ORDERS WITH OSS DATA

   Work Orders with OSS_Order__c: 156
   Work Orders without OSS_Order__c: 3,321
   Unique workorder IDs to query: 156
   Chunk 1: Retrieved 84 workorders

✅ Enriched 84 work orders with OSS data


In [9]:
# === STEP 4: CHECK OSS ACTIVELY BILLING STATUS ===

print("\n" + "=" * 80)
print("STEP 4: CHECKING OSS ACTIVELY BILLING STATUS")
print("=" * 80)

# Identify orders with OSS_Order__c
has_oss_id = orders_after_rt_filter["OSS_Order__c"].notna() & (
    orders_after_rt_filter["OSS_Order__c"] != ""
)
orders_with_oss = orders_after_rt_filter[has_oss_id].copy()
orders_without_oss = orders_after_rt_filter[~has_oss_id].copy()

print(f"\n📊 OSS_Order__c Population:")
print(f"   With OSS_Order__c: {len(orders_with_oss):,}")
print(f"   Without OSS_Order__c: {len(orders_without_oss):,}")

# Query OSS for orders with OSS_Order__c
if len(orders_with_oss) > 0:
    oss_order_ids = (
        orders_with_oss["OSS_Order__c"].dropna().astype(int).unique().tolist()
    )
    print(f"\n   Unique OSS Order IDs to query: {len(oss_order_ids):,}")

    # Query in chunks
    chunk_size = 5000
    oss_orders = []

    for i in range(0, len(oss_order_ids), chunk_size):
        chunk = oss_order_ids[i : i + chunk_size]
        ids_str = ",".join(str(x) for x in chunk)

        oss_query = f"""
        SELECT 
            order_id,
            order_state_cd,
            order_type_cd,
            bill_start_date,
            bill_end_date,
            circuit_active_date,
            account_id,
            service_id
        FROM om.orders
        WHERE order_id IN ({ids_str})
        """

        with oss_conn.cursor(cursor_factory=RealDictCursor) as cur:
            cur.execute(oss_query)
            rows = cur.fetchall()
            oss_orders.extend([dict(row) for row in rows])

        print(f"   Chunk {i//chunk_size + 1}: Retrieved {len(rows)} orders")

    oss_orders_df = pd.DataFrame(oss_orders)
    print(f"\n✅ Total OSS orders retrieved: {len(oss_orders_df):,}")

    # Check for OSS IDs not found
    found_ids = (
        set(oss_orders_df["order_id"].tolist()) if len(oss_orders_df) > 0 else set()
    )
    not_found_ids = set(oss_order_ids) - found_ids
    print(f"   OSS Order IDs not found in OSS: {len(not_found_ids):,}")
else:
    oss_orders_df = pd.DataFrame()
    not_found_ids = set()


STEP 4: CHECKING OSS ACTIVELY BILLING STATUS

📊 OSS_Order__c Population:
   With OSS_Order__c: 11,525
   Without OSS_Order__c: 2,083

   Unique OSS Order IDs to query: 11,518
   Chunk 1: Retrieved 5000 orders
   Chunk 2: Retrieved 4999 orders
   Chunk 3: Retrieved 1518 orders

✅ Total OSS orders retrieved: 11,517
   OSS Order IDs not found in OSS: 1


In [10]:
# === STEP 5: DETERMINE ACTIVELY BILLING STATUS ===

print("\n" + "=" * 80)
print("STEP 5: DETERMINING ACTIVELY BILLING STATUS")
print("=" * 80)

today = pd.Timestamp.now().normalize()
print(f"\nToday's date for comparison: {today.date()}")
print(f"Active OSS States: {ACTIVE_OSS_STATES}")

# Merge SF orders with OSS data
if len(orders_with_oss) > 0:
    orders_with_oss["OSS_Order_ID"] = orders_with_oss["OSS_Order__c"].astype(int)

if len(oss_orders_df) > 0:
    # Convert date columns
    oss_orders_df["bill_start_date"] = pd.to_datetime(
        oss_orders_df["bill_start_date"], utc=True
    ).dt.tz_localize(None)
    oss_orders_df["bill_end_date"] = pd.to_datetime(
        oss_orders_df["bill_end_date"], utc=True
    ).dt.tz_localize(None)

    # Merge
    merged_df = orders_with_oss.merge(
        oss_orders_df, left_on="OSS_Order_ID", right_on="order_id", how="left"
    )
else:
    merged_df = orders_with_oss.copy()
    merged_df["order_id"] = None
    merged_df["order_state_cd"] = None
    merged_df["bill_start_date"] = None
    merged_df["bill_end_date"] = None


# Determine actively billing status
def get_billing_status(row):
    if pd.isna(row.get("order_id")):
        return "No OSS Match"

    state = row["order_state_cd"].strip() if row.get("order_state_cd") else None
    bill_start = row.get("bill_start_date")
    bill_end = row.get("bill_end_date")

    # Check if state is in allowed active states (CL or OA)
    if state not in ACTIVE_OSS_STATES:
        return f"Not Active State ({state})"

    if pd.isna(bill_start):
        return f"{state} - No Bill Start Date"

    if bill_start > today:
        return f"{state} - Bill Start Future"

    if pd.notna(bill_end) and bill_end <= today:
        return f"{state} - Bill End Passed"

    return "ACTIVELY BILLING"


merged_df["OSS_Billing_Status"] = merged_df.apply(get_billing_status, axis=1)

# Add state description
merged_df["OSS_State_Desc"] = merged_df["order_state_cd"].map(OSS_ORDER_STATES)

# Also handle orders without OSS link
orders_without_oss["OSS_Billing_Status"] = "No OSS_Order__c in SF"
orders_without_oss["order_state_cd"] = None
orders_without_oss["OSS_State_Desc"] = None
orders_without_oss["bill_start_date"] = None
orders_without_oss["bill_end_date"] = None

# Summary
print("\n📊 OSS Billing Status Distribution (orders with OSS_Order__c):")
if len(merged_df) > 0:
    billing_status_counts = merged_df["OSS_Billing_Status"].value_counts()
    for status, count in billing_status_counts.items():
        pct = 100 * count / len(merged_df)
        print(f"   {status}: {count:,} ({pct:.1f}%)")


STEP 5: DETERMINING ACTIVELY BILLING STATUS

Today's date for comparison: 2026-01-07
Active OSS States: ['CL', 'OA']

📊 OSS Billing Status Distribution (orders with OSS_Order__c):
   ACTIVELY BILLING: 11,489 (99.7%)
   CL - Bill End Passed: 23 (0.2%)
   Not Active State (CA): 6 (0.1%)
   CL - Bill Start Future: 6 (0.1%)
   No OSS Match: 1 (0.0%)


In [11]:
# === STEP 6: FILTER - ACTIVELY BILLING ONLY ===

print("\n" + "=" * 80)
print("STEP 6: FILTER - ACTIVELY BILLING ONLY")
print("=" * 80)

# Get common columns
common_cols = [col for col in orders_after_rt_filter.columns]
oss_cols = [
    "order_state_cd",
    "OSS_State_Desc",
    "bill_start_date",
    "bill_end_date",
    "OSS_Billing_Status",
]

# Ensure all columns exist
for col in oss_cols:
    if col not in merged_df.columns:
        merged_df[col] = None
    if col not in orders_without_oss.columns:
        orders_without_oss[col] = None

all_cols = common_cols + oss_cols

# Combine orders with and without OSS
all_checked_orders = pd.concat(
    [
        merged_df[[c for c in all_cols if c in merged_df.columns]],
        orders_without_oss[[c for c in all_cols if c in orders_without_oss.columns]],
    ],
    ignore_index=True,
)

# Split into actively billing vs not
actively_billing_mask = all_checked_orders["OSS_Billing_Status"] == "ACTIVELY BILLING"
orders_actively_billing = all_checked_orders[actively_billing_mask].copy()
excluded_not_billing_df = all_checked_orders[~actively_billing_mask].copy()

print(f"\n📊 Actively Billing Filter:")
print(f"   Before: {len(all_checked_orders):,}")
print(f"   ❌ Excluded (Not Actively Billing): {len(excluded_not_billing_df):,}")
print(f"   ✅ Confirmed Actively Billing: {len(orders_actively_billing):,}")

# Show breakdown of excluded
if len(excluded_not_billing_df) > 0:
    print(f"\n   Breakdown of excluded:")
    for status, count in (
        excluded_not_billing_df["OSS_Billing_Status"].value_counts().items()
    ):
        print(f"      {status}: {count:,}")

# Set this as our active orders for the rest of the pipeline
active_orders_df = orders_actively_billing.copy()


STEP 6: FILTER - ACTIVELY BILLING ONLY

📊 Actively Billing Filter:
   Before: 13,608
   ❌ Excluded (Not Actively Billing): 2,119
   ✅ Confirmed Actively Billing: 11,489

   Breakdown of excluded:
      No OSS_Order__c in SF: 2,083
      CL - Bill End Passed: 23
      Not Active State (CA): 6
      CL - Bill Start Future: 6
      No OSS Match: 1


C:\Users\vjero\AppData\Local\Temp\ipykernel_6256\1019780369.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_checked_orders = pd.concat(
C:\Users\vjero\AppData\Local\Temp\ipykernel_6256\1019780369.py:27: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_checked_orders = pd.concat(


In [12]:
# === STEP 7: GET NEW BBF BANS ===

print("\n" + "=" * 80)
print("STEP 7: QUERYING NEW BBF BANS")
print("=" * 80)

bbf_ban_query = """
SELECT 
    Id,
    Name,
    Account__c,
    Account__r.Name,
    Legacy_ES_Id__c,
    BBF_Ban__c,
    Billing_Address_1__c,
    Billing_City__c,
    Billing_State__c,
    Billing_ZIP__c,
    Payment_Terms__c,
    Active_Billing__c
FROM Billing_Invoice__c
WHERE BBF_Ban__c = true
  AND Legacy_ES_Id__c != null
"""

print("Querying new BBF BANs (BBF_Ban__c = true, Legacy_ES_Id__c populated)...")
result = es_sf.query_all(bbf_ban_query)
bbf_bans_df = pd.DataFrame(result["records"])

if len(bbf_bans_df) > 0:
    # Flatten Account name
    if "Account__r" in bbf_bans_df.columns:
        bbf_bans_df["Account_Name"] = bbf_bans_df["Account__r"].apply(
            lambda x: x["Name"] if x else None
        )

    # Clean up
    bbf_bans_df = bbf_bans_df.drop(
        columns=["attributes", "Account__r"], errors="ignore"
    )

    print(f"\n✅ Found {len(bbf_bans_df):,} new BBF BANs")

    # Create lookup from legacy BAN ID to new BBF BAN
    legacy_to_bbf_ban = {}
    for _, ban in bbf_bans_df.iterrows():
        legacy_id = ban.get("Legacy_ES_Id__c")
        if legacy_id:
            legacy_to_bbf_ban[legacy_id] = {
                "Id": ban["Id"],
                "Name": ban["Name"],
                "Account__c": ban.get("Account__c"),
                "Account_Name": ban.get("Account_Name"),
            }
    print(f"   Legacy BAN to BBF BAN mappings: {len(legacy_to_bbf_ban):,}")
else:
    legacy_to_bbf_ban = {}
    print("⚠️ No BBF BANs found")


STEP 7: QUERYING NEW BBF BANS
Querying new BBF BANs (BBF_Ban__c = true, Legacy_ES_Id__c populated)...

✅ Found 2,505 new BBF BANs
   Legacy BAN to BBF BAN mappings: 2,505


In [13]:
# === STEP 8: MAP ORDERS TO BBF BANS ===

print("\n" + "=" * 80)
print("STEP 8: MAPPING ORDERS TO BBF BANS")
print("=" * 80)

# Track which orders can be mapped
orders_with_bbf_ban = []
orders_missing_bbf_ban = []
orders_missing_any_ban = []

for _, order in active_orders_df.iterrows():
    legacy_ban_id = order.get("Billing_Invoice__c")
    order_dict = order.to_dict()

    if not legacy_ban_id:
        # Order has no BAN at all
        orders_missing_any_ban.append(order_dict)
    elif legacy_ban_id in legacy_to_bbf_ban:
        # Order can be mapped to new BBF BAN
        bbf_ban = legacy_to_bbf_ban[legacy_ban_id]
        order_dict["New_BBF_BAN_Id"] = bbf_ban["Id"]
        order_dict["New_BBF_BAN_Name"] = bbf_ban["Name"]
        order_dict["New_BBF_BAN_Account__c"] = bbf_ban["Account__c"]
        order_dict["New_BBF_BAN_Account_Name"] = bbf_ban["Account_Name"]
        orders_with_bbf_ban.append(order_dict)
    else:
        # Order has legacy BAN but no BBF BAN mapping
        orders_missing_bbf_ban.append(order_dict)

orders_ready_df = pd.DataFrame(orders_with_bbf_ban)
orders_no_bbf_ban_df = pd.DataFrame(orders_missing_bbf_ban)
orders_no_ban_df = pd.DataFrame(orders_missing_any_ban)

print(f"\n📊 BAN Mapping Results:")
print(f"   ✅ Ready to migrate (has BBF BAN): {len(orders_ready_df):,}")
print(f"   ⚠️ Missing BBF BAN mapping: {len(orders_no_bbf_ban_df):,}")
print(f"   ❌ Missing ANY BAN: {len(orders_no_ban_df):,}")


STEP 8: MAPPING ORDERS TO BBF BANS

📊 BAN Mapping Results:
   ✅ Ready to migrate (has BBF BAN): 11,475
   ⚠️ Missing BBF BAN mapping: 12
   ❌ Missing ANY BAN: 2


In [14]:
# === STEP 9: IDENTIFY ACCOUNTS TO MIGRATE ===

print("\n" + "=" * 80)
print("STEP 9: IDENTIFYING ACCOUNTS TO MIGRATE")
print("=" * 80)

if len(orders_ready_df) > 0:
    unique_account_ids = (
        orders_ready_df["New_BBF_BAN_Account__c"].dropna().unique().tolist()
    )
    print(f"\nFound {len(unique_account_ids):,} unique Accounts from new BBF BANs")

    if unique_account_ids:
        chunk_size = 150
        all_accounts = []

        for i in range(0, len(unique_account_ids), chunk_size):
            chunk = unique_account_ids[i : i + chunk_size]
            ids_str = "','".join(chunk)

            account_query = f"""
            SELECT Id, Name, Type, Industry, 
                   BillingStreet, BillingCity, BillingState, BillingPostalCode, BillingCountry,
                   Phone, Website, BBF_New_Id__c
            FROM Account
            WHERE Id IN ('{ids_str}')
            """
            result = es_sf.query_all(account_query)
            all_accounts.extend(result["records"])

        accounts_df = pd.DataFrame(all_accounts)
        if len(accounts_df) > 0:
            accounts_df = accounts_df.drop(columns=["attributes"], errors="ignore")
        print(f"\n✅ Accounts to migrate: {len(accounts_df):,}")
    else:
        accounts_df = pd.DataFrame()
else:
    accounts_df = pd.DataFrame()
    print("⚠️ No orders ready to migrate, skipping Account query")


STEP 9: IDENTIFYING ACCOUNTS TO MIGRATE

Found 2,224 unique Accounts from new BBF BANs

✅ Accounts to migrate: 2,224


In [15]:
# === STEP 10: IDENTIFY CONTACTS TO MIGRATE ===

print("\n" + "=" * 80)
print("STEP 10: IDENTIFYING CONTACTS TO MIGRATE")
print("=" * 80)

if len(accounts_df) > 0:
    account_ids = accounts_df["Id"].tolist()
    chunk_size = 150
    all_contacts = []

    for i in range(0, len(account_ids), chunk_size):
        chunk = account_ids[i : i + chunk_size]
        ids_str = "','".join(chunk)

        contact_query = f"""
        SELECT Id, AccountId, FirstName, LastName, Email, Phone, Title,
               MailingStreet, MailingCity, MailingState, MailingPostalCode,
               BBF_New_Id__c
        FROM Contact
        WHERE AccountId IN ('{ids_str}')
        """
        result = es_sf.query_all(contact_query)
        all_contacts.extend(result["records"])

    contacts_df = pd.DataFrame(all_contacts)
    if len(contacts_df) > 0:
        contacts_df = contacts_df.drop(columns=["attributes"], errors="ignore")
    print(f"\n✅ Contacts to migrate: {len(contacts_df):,}")
else:
    contacts_df = pd.DataFrame()
    print("⚠️ No accounts to migrate, skipping Contact query")


STEP 10: IDENTIFYING CONTACTS TO MIGRATE

✅ Contacts to migrate: 15,576


In [16]:
# === STEP 11: IDENTIFY LOCATIONS TO MIGRATE ===

print("\n" + "=" * 80)
print("STEP 11: IDENTIFYING LOCATIONS TO MIGRATE")
print("=" * 80)

if len(orders_ready_df) > 0:
    address_a_ids = orders_ready_df["Address_A__c"].dropna().unique().tolist()
    address_z_ids = orders_ready_df["Address_Z__c"].dropna().unique().tolist()
    all_address_ids = list(set(address_a_ids + address_z_ids))

    print(f"\nUnique locations referenced by orders:")
    print(f"   Address_A: {len(address_a_ids):,}")
    print(f"   Address_Z: {len(address_z_ids):,}")
    print(f"   Combined unique: {len(all_address_ids):,}")

    if all_address_ids:
        chunk_size = 150
        all_locations = []

        for i in range(0, len(all_address_ids), chunk_size):
            chunk = all_address_ids[i : i + chunk_size]
            ids_str = "','".join(chunk)

            location_query = f"""
            SELECT Id, Name, Address__c, City__c, State__c, County__c, Zip__c,
                   Complete_Address__c, CLLI__c, Building_Status__c, On_Net__c,
                   BBF_New_Id__c
            FROM Address__c
            WHERE Id IN ('{ids_str}')
            """
            result = es_sf.query_all(location_query)
            all_locations.extend(result["records"])

        locations_df = pd.DataFrame(all_locations)
        if len(locations_df) > 0:
            locations_df = locations_df.drop(columns=["attributes"], errors="ignore")
        print(f"\n✅ Locations to migrate: {len(locations_df):,}")
    else:
        locations_df = pd.DataFrame()
else:
    locations_df = pd.DataFrame()
    all_address_ids = []
    print("⚠️ No orders ready to migrate, skipping Location query")


STEP 11: IDENTIFYING LOCATIONS TO MIGRATE

Unique locations referenced by orders:
   Address_A: 723
   Address_Z: 9,818
   Combined unique: 10,175

✅ Locations to migrate: 10,175


In [17]:
# === STEP 12: IDENTIFY OFF_NET RECORDS TO MIGRATE ===

print("\n" + "=" * 80)
print("STEP 12: IDENTIFYING OFF_NET RECORDS TO MIGRATE")
print("=" * 80)

if len(all_address_ids) > 0:
    chunk_size = 100
    all_offnet = []

    for i in range(0, len(all_address_ids), chunk_size):
        chunk = all_address_ids[i : i + chunk_size]
        ids_str = "','".join(chunk)

        offnet_query = f"""
        SELECT Id, Name, 
               Location_1__c, Location_1_Address__c,
               Location_2__c, Location_2_Address__c,
               Off_Net_Vendor__c, Vendor_Name__c,
               Vendor_circuit_Id__c, Internal_Circuit_Id__c,
               Cost_MRC__c, Cost_NRC__c, Invoice_MRC__c,
               LEC_Order_Status__c, Off_Net_Type__c,
               Bandwidth__c, Circuit_Type__c, Term__c,
               Term_Agreement_Start_Date__c, Term_Agreement_End_Date__c,
               Vendor_Bill_Start_Date__c, Vendor_Bill_Stop_Date__c,
               SOF1__c
        FROM Off_Net__c
        WHERE Location_1__c IN ('{ids_str}')
           OR Location_2__c IN ('{ids_str}')
        """
        result = es_sf.query_all(offnet_query)
        all_offnet.extend(result["records"])

    offnet_df = pd.DataFrame(all_offnet)
    if len(offnet_df) > 0:
        offnet_df = offnet_df.drop(columns=["attributes"], errors="ignore")
        offnet_df = offnet_df.drop_duplicates(subset=["Id"])
    print(f"\n✅ Off_Net records to migrate: {len(offnet_df):,}")
else:
    offnet_df = pd.DataFrame()
    print("⚠️ No locations to migrate, skipping Off_Net query")


STEP 12: IDENTIFYING OFF_NET RECORDS TO MIGRATE

✅ Off_Net records to migrate: 2,157


In [18]:
# === STEP 13: DATA QUALITY ANALYSIS ===

print("\n" + "=" * 80)
print("STEP 13: DATA QUALITY ANALYSIS")
print("=" * 80)

data_quality_issues = []

if len(orders_ready_df) > 0:
    total_ready = len(orders_ready_df)

    # Check for missing Address_A__c
    missing_addr_a = orders_ready_df["Address_A__c"].isna().sum()
    if missing_addr_a > 0:
        data_quality_issues.append(
            {
                "Issue": "Orders missing Address_A__c",
                "Count": missing_addr_a,
                "Percentage": f"{missing_addr_a/total_ready*100:.1f}%",
                "Severity": "HIGH",
                "Impact": "Cannot set A_Location__c on BBF Service__c",
            }
        )

    # Check for missing Node__c
    missing_node = orders_ready_df["Node__c"].isna().sum()
    if missing_node > 0:
        data_quality_issues.append(
            {
                "Issue": "Orders missing Node__c",
                "Count": missing_node,
                "Percentage": f"{missing_node/total_ready*100:.1f}%",
                "Severity": "LOW",
                "Impact": "Can fix post-migration - A_Node__c/Z_Node__c optional",
            }
        )

    # Check for missing Service_Start_Date__c - but OSS has bill_start_date
    sf_start_null = orders_ready_df["Service_Start_Date__c"].isna()
    oss_start_exists = orders_ready_df["bill_start_date"].notna()
    missing_sf_has_oss = (sf_start_null & oss_start_exists).sum()

    if sf_start_null.sum() > 0:
        data_quality_issues.append(
            {
                "Issue": "Orders missing Service_Start_Date__c in SF",
                "Count": sf_start_null.sum(),
                "Percentage": f"{sf_start_null.sum()/total_ready*100:.1f}%",
                "Severity": "LOW",
                "Impact": f"OSS has bill_start_date for {missing_sf_has_oss:,} - USE FOR MIGRATION",
            }
        )

data_quality_df = pd.DataFrame(data_quality_issues)

if len(data_quality_issues) > 0:
    print("\n📊 Data Quality Issues:")
    for issue in data_quality_issues:
        print(
            f"   [{issue['Severity']}] {issue['Issue']}: {issue['Count']} ({issue['Percentage']})"
        )
        print(f"         → {issue['Impact']}")
else:
    print("\n✅ No significant data quality issues found")


STEP 13: DATA QUALITY ANALYSIS

📊 Data Quality Issues:
   [HIGH] Orders missing Address_A__c: 2 (0.0%)
         → Cannot set A_Location__c on BBF Service__c
   [LOW] Orders missing Node__c: 9784 (85.3%)
         → Can fix post-migration - A_Node__c/Z_Node__c optional
   [LOW] Orders missing Service_Start_Date__c in SF: 4425 (38.6%)
         → OSS has bill_start_date for 4,425 - USE FOR MIGRATION


In [19]:
# === STEP 14: CREATE BAN MAPPING TABLE ===

print("\n" + "=" * 80)
print("STEP 14: CREATING BAN MAPPING TABLE")
print("=" * 80)

if len(orders_ready_df) > 0:
    ban_mapping_data = []
    legacy_ban_counts = orders_ready_df.groupby("Billing_Invoice__c").size().to_dict()

    for legacy_id, bbf_ban in legacy_to_bbf_ban.items():
        order_count = legacy_ban_counts.get(legacy_id, 0)
        if order_count > 0:
            ban_mapping_data.append(
                {
                    "Legacy_BAN_Id": legacy_id,
                    "New_BBF_BAN_Id": bbf_ban["Id"],
                    "New_BBF_BAN_Name": bbf_ban["Name"],
                    "Account__c": bbf_ban["Account__c"],
                    "Account_Name": bbf_ban["Account_Name"],
                    "Order_Count": order_count,
                }
            )

    ban_mapping_df = pd.DataFrame(ban_mapping_data)
    ban_mapping_df = ban_mapping_df.sort_values("Order_Count", ascending=False)
    print(f"\n✅ BAN mappings with orders: {len(ban_mapping_df):,}")
else:
    ban_mapping_df = pd.DataFrame()


STEP 14: CREATING BAN MAPPING TABLE

✅ BAN mappings with orders: 2,440


In [20]:
# === STEP 15: GENERATE SUMMARY ===

print("\n" + "=" * 80)
print("STEP 15: MIGRATION SUMMARY")
print("=" * 80)

summary_data = [
    {"Category": "FILTER PIPELINE", "Metric": "", "Count": "", "Notes": ""},
    {
        "Category": "",
        "Metric": "1. Total Active Status Orders",
        "Count": len(all_orders_df),
        "Notes": "All orders with qualifying status",
    },
    {
        "Category": "",
        "Metric": "2. Excluded (PA MARKET DECOM)",
        "Count": len(excluded_pa_decom_df),
        "Notes": "Project_Group__c contains 'PA MARKET DECOM'",
    },
    {
        "Category": "",
        "Metric": "3. Excluded (Work Orders)",
        "Count": len(excluded_work_orders_df),
        "Notes": f"Service_Order_Record_Type__c != '{VALID_RECORD_TYPE}'",
    },
    {
        "Category": "",
        "Metric": "4. Excluded (Not Actively Billing)",
        "Count": len(excluded_not_billing_df),
        "Notes": f"OSS: not in {ACTIVE_OSS_STATES} or billing dates invalid",
    },
    {
        "Category": "",
        "Metric": "5. Confirmed Actively Billing",
        "Count": len(active_orders_df),
        "Notes": "Passed all filters",
    },
    {"Category": "", "Metric": "", "Count": "", "Notes": ""},
    {"Category": "BAN MAPPING", "Metric": "", "Count": "", "Notes": ""},
    {
        "Category": "",
        "Metric": "Ready (has BBF BAN)",
        "Count": len(orders_ready_df),
        "Notes": "Can be migrated now",
    },
    {
        "Category": "",
        "Metric": "Missing BBF BAN",
        "Count": len(orders_no_bbf_ban_df),
        "Notes": "Need new BBF BAN created",
    },
    {
        "Category": "",
        "Metric": "Missing ANY BAN",
        "Count": len(orders_no_ban_df),
        "Notes": "CRITICAL - no BAN reference",
    },
    {"Category": "", "Metric": "", "Count": "", "Notes": ""},
    {"Category": "RECORDS TO MIGRATE", "Metric": "", "Count": "", "Notes": ""},
    {
        "Category": "",
        "Metric": "BANs (with orders)",
        "Count": len(ban_mapping_df),
        "Notes": "Billing_Invoice__c to BAN__c",
    },
    {
        "Category": "",
        "Metric": "Accounts",
        "Count": len(accounts_df),
        "Notes": "Unique accounts from BBF BANs",
    },
    {
        "Category": "",
        "Metric": "Contacts",
        "Count": len(contacts_df),
        "Notes": "Contacts for migration accounts",
    },
    {
        "Category": "",
        "Metric": "Locations",
        "Count": len(locations_df),
        "Notes": "Address_A + Address_Z from orders",
    },
    {
        "Category": "",
        "Metric": "Off_Net",
        "Count": len(offnet_df),
        "Notes": "Off_Net for migration locations",
    },
]

summary_df = pd.DataFrame(summary_data)

print(
    f"""
📊 SUMMARY
{'─'*50}
FILTER PIPELINE:
  Total Active Status Orders:     {len(all_orders_df):,}
  ├─ Excluded (PA MARKET DECOM):  {len(excluded_pa_decom_df):,}
  ├─ Excluded (Work Orders):      {len(excluded_work_orders_df):,}
  ├─ Excluded (Not Billing OSS):  {len(excluded_not_billing_df):,}
  └─ Confirmed Actively Billing:  {len(active_orders_df):,}

BAN MAPPING:
  ├─ Ready to migrate:            {len(orders_ready_df):,}
  ├─ Missing BBF BAN:             {len(orders_no_bbf_ban_df):,}
  └─ Missing ANY BAN:             {len(orders_no_ban_df):,}

RECORDS TO MIGRATE:
  ├─ BANs:                        {len(ban_mapping_df):,}
  ├─ Accounts:                    {len(accounts_df):,}
  ├─ Contacts:                    {len(contacts_df):,}
  ├─ Locations:                   {len(locations_df):,}
  └─ Off_Net:                     {len(offnet_df):,}
"""
)


STEP 15: MIGRATION SUMMARY

📊 SUMMARY
──────────────────────────────────────────────────
FILTER PIPELINE:
  Total Active Status Orders:     17,972
  ├─ Excluded (PA MARKET DECOM):  887
  ├─ Excluded (Work Orders):      3,477
  ├─ Excluded (Not Billing OSS):  2,119
  └─ Confirmed Actively Billing:  11,489

BAN MAPPING:
  ├─ Ready to migrate:            11,475
  ├─ Missing BBF BAN:             12
  └─ Missing ANY BAN:             2

RECORDS TO MIGRATE:
  ├─ BANs:                        2,440
  ├─ Accounts:                    2,224
  ├─ Contacts:                    15,576
  ├─ Locations:                   10,175
  └─ Off_Net:                     2,157



In [21]:
# === STEP 16: EXPORT TO EXCEL ===

print("\n" + "=" * 80)
print("STEP 16: EXPORTING TO EXCEL")
print("=" * 80)

wb = Workbook()
ws_summary = wb.active
ws_summary.title = "Summary"

# Styles
header_font = Font(bold=True, size=12, color="FFFFFF")
header_fill = PatternFill(start_color="4472C4", end_color="4472C4", fill_type="solid")
thin_border = Border(
    left=Side(style="thin"),
    right=Side(style="thin"),
    top=Side(style="thin"),
    bottom=Side(style="thin"),
)


def write_df_to_sheet(ws, df, start_row=1):
    """Write dataframe to worksheet with formatting"""
    for r_idx, row in enumerate(
        dataframe_to_rows(df, index=False, header=True), start=start_row
    ):
        for c_idx, value in enumerate(row, start=1):
            cell = ws.cell(row=r_idx, column=c_idx, value=value)
            if r_idx == start_row:
                cell.font = header_font
                cell.fill = header_fill
            cell.border = thin_border


# --- SHEET 1: Summary ---
ws_summary.append(["ES → BBF Migration Analysis (v3)"])
ws_summary["A1"].font = Font(bold=True, size=16)
ws_summary.append([f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"])
ws_summary.append(
    [
        f"OSS Active States: {ACTIVE_OSS_STATES} | Record Type filter | OSS Billing validation"
    ]
)
ws_summary.append([])
write_df_to_sheet(ws_summary, summary_df, start_row=5)

ws_summary.column_dimensions["A"].width = 20
ws_summary.column_dimensions["B"].width = 40
ws_summary.column_dimensions["C"].width = 12
ws_summary.column_dimensions["D"].width = 55
print("   ✅ Summary")

# --- SHEET 2: Active Orders (Ready) ---
if len(orders_ready_df) > 0:
    ws = wb.create_sheet("Active_Orders")
    export_cols = [
        "Id",
        "Name",
        "Service_ID__c",
        "Status",
        "Account_Name",
        "Billing_Invoice__c",
        "New_BBF_BAN_Id",
        "New_BBF_BAN_Name",
        "Address_A__c",
        "Address_Z__c",
        "SOF_MRC__c",
        "Service_Start_Date__c",
        "OSS_Order__c",
        "order_state_cd",
        "bill_start_date",
        "bill_end_date",
    ]
    export_cols = [c for c in export_cols if c in orders_ready_df.columns]
    write_df_to_sheet(ws, orders_ready_df[export_cols])
    print(f"   ✅ Active_Orders ({len(orders_ready_df):,} records)")

# --- SHEET 3: BAN Mapping ---
if len(ban_mapping_df) > 0:
    ws = wb.create_sheet("BAN_Mapping")
    write_df_to_sheet(ws, ban_mapping_df)
    print(f"   ✅ BAN_Mapping ({len(ban_mapping_df):,} records)")

# --- SHEET 4: Accounts ---
if len(accounts_df) > 0:
    ws = wb.create_sheet("Accounts")
    write_df_to_sheet(ws, accounts_df)
    print(f"   ✅ Accounts ({len(accounts_df):,} records)")

# --- SHEET 5: Contacts ---
if len(contacts_df) > 0:
    ws = wb.create_sheet("Contacts")
    write_df_to_sheet(ws, contacts_df)
    print(f"   ✅ Contacts ({len(contacts_df):,} records)")

# --- SHEET 6: Locations ---
if len(locations_df) > 0:
    ws = wb.create_sheet("Locations")
    write_df_to_sheet(ws, locations_df)
    print(f"   ✅ Locations ({len(locations_df):,} records)")

# --- SHEET 7: Off_Net ---
if len(offnet_df) > 0:
    ws = wb.create_sheet("Off_Net")
    write_df_to_sheet(ws, offnet_df)
    print(f"   ✅ Off_Net ({len(offnet_df):,} records)")

# --- SHEET 8: Data Quality ---
if len(data_quality_df) > 0:
    ws = wb.create_sheet("Data_Quality")
    write_df_to_sheet(ws, data_quality_df)
    print(f"   ✅ Data_Quality ({len(data_quality_df):,} issues)")

# --- SHEET 9: Orders Missing BBF BAN ---
if len(orders_no_bbf_ban_df) > 0:
    ws = wb.create_sheet("Orders_Missing_BBF_BAN")
    export_cols = [
        "Id",
        "Name",
        "Service_ID__c",
        "Status",
        "Account_Name",
        "Billing_Invoice__c",
        "SOF_MRC__c",
    ]
    export_cols = [c for c in export_cols if c in orders_no_bbf_ban_df.columns]
    write_df_to_sheet(ws, orders_no_bbf_ban_df[export_cols])
    print(f"   ✅ Orders_Missing_BBF_BAN ({len(orders_no_bbf_ban_df):,} records)")

# --- SHEET 10: Excluded - PA MARKET DECOM ---
if len(excluded_pa_decom_df) > 0:
    ws = wb.create_sheet("Excluded_PA_DECOM")
    export_cols = [
        "Id",
        "Name",
        "Service_ID__c",
        "Status",
        "Project_Group__c",
        "Account_Name",
    ]
    export_cols = [c for c in export_cols if c in excluded_pa_decom_df.columns]
    write_df_to_sheet(ws, excluded_pa_decom_df[export_cols])
    print(f"   ✅ Excluded_PA_DECOM ({len(excluded_pa_decom_df):,} records)")

# --- SHEET 11: Excluded - Work Orders (enriched with OSS data) ---
if len(excluded_work_orders_df) > 0:
    ws = wb.create_sheet("Excluded_Work_Orders")
    export_cols = [
        "Id",
        "Name",
        "Service_ID__c",
        "Status",
        "Service_Order_Record_Type__c",
        "Account_Name",
        "OSS_Order__c",
        "order_nm",
        "order_id",
        "workorder_type_cd",
        "workorder_type_desc",
        "workorder_state_cd",
        "workorder_state_desc",
        "description",
        "start_date",
        "end_date",
    ]
    export_cols = [c for c in export_cols if c in excluded_work_orders_df.columns]
    write_df_to_sheet(ws, excluded_work_orders_df[export_cols])
    print(f"   ✅ Excluded_Work_Orders ({len(excluded_work_orders_df):,} records)")

# --- SHEET 12: Excluded - Not Actively Billing ---
if len(excluded_not_billing_df) > 0:
    ws = wb.create_sheet("Excluded_Not_Billing")
    export_cols = [
        "Id",
        "Name",
        "Service_ID__c",
        "Status",
        "Account_Name",
        "OSS_Order__c",
        "order_state_cd",
        "OSS_State_Desc",
        "bill_start_date",
        "bill_end_date",
        "OSS_Billing_Status",
    ]
    export_cols = [c for c in export_cols if c in excluded_not_billing_df.columns]
    write_df_to_sheet(ws, excluded_not_billing_df[export_cols])
    print(f"   ✅ Excluded_Not_Billing ({len(excluded_not_billing_df):,} records)")

# Save
wb.save(OUTPUT_FILE)
print(f"\n✅ Excel file saved: {OUTPUT_FILE}")


STEP 16: EXPORTING TO EXCEL
   ✅ Summary
   ✅ Active_Orders (11,475 records)
   ✅ BAN_Mapping (2,440 records)
   ✅ Accounts (2,224 records)
   ✅ Contacts (15,576 records)
   ✅ Locations (10,175 records)
   ✅ Off_Net (2,157 records)
   ✅ Data_Quality (3 issues)
   ✅ Orders_Missing_BBF_BAN (12 records)
   ✅ Excluded_PA_DECOM (887 records)
   ✅ Excluded_Work_Orders (3,477 records)
   ✅ Excluded_Not_Billing (2,119 records)


TypeError: Excel does not support timezones in datetimes. The tzinfo in the datetime/time object must be set to None.

In [ ]:
# === CLEANUP ===

print("\n" + "=" * 80)
print("CLEANUP")
print("=" * 80)

oss_conn.close()
print("✅ OSS connection closed")

print(f"\n" + "=" * 80)
print("ANALYSIS COMPLETE")
print("=" * 80)
print(f"\nOutput file: {OUTPUT_FILE}")

---
## v3 Changes Summary

### OSS Active States
- Now includes both **CL (Closed)** and **OA (Accepted)** as valid active states
- Both must still pass billing date criteria (bill_start_date <= today, bill_end_date null or > today)

### Data Quality - Service Start Date
- Updated to show that OSS `bill_start_date` can be used when SF `Service_Start_Date__c` is missing
- Severity reduced to LOW since we have the data from OSS

### Work Orders Enrichment
- Excluded_Work_Orders sheet now includes OSS data from `workorders.workorders` table:
  - `order_nm`, `order_id`, `workorder_type_cd`, `workorder_state_cd`, `description`, `start_date`, `end_date`
- Only includes active workorders (`disabled = 'infinity'`)

### Next Steps
Same as v2 - proceed with migration execution order.